In [1]:
from collections import Counter
import gc
import itertools
from pathlib import Path
import re
import time

import numpy as np
import pandas as pd
import gensim

from tqdm import tqdm
tqdm.pandas()

In [2]:
data_dir = Path.home() / 'Desktop/kaggle/data/quora'
train_all = pd.read_csv(data_dir / 'train.csv.zip')
print("Train shape:", train_all.shape)

Train shape: (1306122, 3)


In [3]:
def build_vocab(sentences, verbose =  True):
    vocab = Counter()
    for sentence in tqdm(sentences, disable=(not verbose)):
        for word in sentence:
            vocab[word] += 1
    return vocab

In [4]:
sentences1 = train_all.question_text.progress_apply(lambda x: x.split()).values
vocab1 = build_vocab(sentences1)

100%|██████████| 1306122/1306122 [00:05<00:00, 226009.66it/s]


In [5]:
vocab1.most_common(10)

[('the', 653948),
 ('What', 417802),
 ('to', 403183),
 ('a', 402682),
 ('in', 363131),
 ('is', 331647),
 ('of', 330825),
 ('I', 306261),
 ('How', 261930),
 ('and', 251973)]

In [6]:
data_dir.open?

In [7]:
PARAGRAM_PATH = data_dir / 'glove.840B.300d/glove.840B.300d.txt'
PARAGRAM_NUM_LINES = 2196016

def key_value_pair(word, *vec):
    return word, np.asarray(vec, dtype='float32')

with PARAGRAM_PATH.open(encoding='utf8') as f:
    embeddings_index = dict(key_value_pair(*line.split(' '))
                            for line in tqdm(f, total=PARAGRAM_NUM_LINES))

2196017it [01:38, 22281.63it/s]                             


In [8]:
list(itertools.islice(embeddings_index.keys(), 10))

['1/21/00',
 'Girl.wmv',
 'appbackr',
 'pposed',
 '17:29:17',
 '182.20',
 'Muyinga',
 'dichotomised',
 'Eminence',
 '10-23-09']

In [9]:
def check_coverage(vocab, embeddings_index):
    covered_vocab = set()
    oov = Counter()
    covered_count = 0
    all_count = 0
    for word, count in tqdm(vocab.items()):
        all_count += count
        if word in embeddings_index:
            covered_vocab.add(word)
            covered_count += count
        else:
            oov[word] = count

    print('Found embeddings for {:.2%} of vocab'.format(len(covered_vocab) / len(vocab)))
    print('Found embeddings for  {:.2%} of all words'.format(covered_count / all_count))
    return oov

In [10]:
oov1 = check_coverage(vocab1, embeddings_index)

100%|██████████| 508823/508823 [00:00<00:00, 1110828.60it/s]

Found embeddings for 33.16% of vocab
Found embeddings for  88.16% of all words


In [11]:
oov1.most_common(10)

[('India?', 16384),
 ('it?', 12900),
 ("What's", 12425),
 ('do?', 8753),
 ('life?', 7753),
 ('you?', 6295),
 ('me?', 6202),
 ('them?', 6140),
 ('time?', 5716),
 ('world?', 5386)]

In [12]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

cnt = 0
for word in embeddings_index.keys():
    if len(word) != 1:
        continue
    if not is_ascii(word):
        continue
    if not re.fullmatch(r'[^a-zA-Z0-9]+', word):
        continue
    print(word, end=' ')
    cnt += 1
    if cnt > 1000:
        break        

\ ] . ? [ / { : } # = - ; " < & ~ ^ > ` | ' _ @ % * , $ + ) ! ( 

In [13]:
punct_beginning = re.compile(r'(^|\b|\s)([("])')
punct_ending = re.compile(r'([.,?)])($|\b|\s)')
punct_covered = re.compile(r'(^|\b|\s)([&./!<}+?)_(;~@*#:={$`|>^",%\[\]\'\\-])($|\b|\s|\?$)')

def clean_text(x):
    x = str(x)
    x = x.replace("’", "'")
    x = x.replace("“", '"')
    x = punct_beginning.sub(lambda m: ' {} '.format(m.group(2)), x)
    x = punct_ending.sub(lambda m: ' {} '.format(m.group(1)), x)
    x = punct_covered.sub(lambda m: ' {} '.format(m.group(2)), x)
    return x

In [14]:
for s in ['a+b', 'aa+bb+c', 'a++b', 'aa?', ' (I']:
    print(punct_covered.sub(lambda m: ' {} '.format(m.group(2)), s))

a + b
aa + bb + c
a++b
aa ? 
 ( I


In [15]:
def split2(text):
    return clean_text(text).split()

In [16]:
sentences2 = train_all.question_text.progress_apply(split2).values
vocab2 = build_vocab(sentences2)
oov2 = check_coverage(vocab2, embeddings_index)

100%|██████████| 252906/252906 [00:00<00:00, 1109555.11it/s]

Found embeddings for 70.77% of vocab
Found embeddings for  99.48% of all words


In [17]:
oov2.most_common(30)

[('Quorans', 856),
 ('Brexit', 493),
 ('cryptocurrencies', 481),
 ('Redmi', 379),
 ('etc.)', 338),
 ('[/math', 208),
 ('^2', 144),
 ('OnePlus', 125),
 ('UCEED', 123),
 ('Blockchain', 111),
 ('GDPR', 107),
 ('demonetisation', 106),
 ('Coinbase', 105),
 ('BNBR', 99),
 ('Adityanath', 99),
 ('Machedo', 99),
 ('Boruto', 93),
 ('ethereum', 89),
 ('DCEU', 89),
 ('IIEST', 85),
 ('\\frac', 81),
 ('SJWs', 79),
 ('Qoura', 79),
 ('Upwork', 70),
 ('LNMIIT', 67),
 ('Kavalireddi', 65),
 ('Zerodha', 65),
 ('bhakts', 63),
 ('Doklam', 61),
 ('Vajiram', 59)]

In [18]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

cnt = 0
for word in embeddings_index.vocab:
    if len(word) not in [1]:
        continue
    if not is_ascii(word):
        continue
    if not re.fullmatch(r'[^a-zA-Z0-9]+', word):
        continue
    print(word, end=' ')
    cnt += 1
    if cnt > 100:
        break        

AttributeError: 'dict' object has no attribute 'vocab'